In [3]:
!pip install ratelimit

In [4]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import time
from tqdm import tqdm

import google.generativeai as genai
from ratelimit import limits, sleep_and_retry

logs_folder = "logs"
models_folder = "models"
data_folder = "data"
broken_data_folder = "broken_data"

for folder in [logs_folder, models_folder, data_folder, broken_data_folder]:
    if not os.path.exists(folder):
        os.makedirs(folder)


In [5]:
import numpy as np

class CoderAgent:
    def __init__(self, num_states, num_actions, epsilon=0.75, alpha=0.5, gamma=0.8):
        self.q_table = np.zeros((num_states, num_actions))
        self.epsilon = epsilon
        self.alpha = alpha
        self.gamma = gamma 
        self.actions = [
            "Remover linhas com valores nulos em 'df'",
            "Preencher valores nulos em colunas numéricas de 'df' com a média",
            "Preencher valores nulos em 'df' com zero",
            "Remover colunas com muitos valores nulos em 'df'",
        ]
        self.action_counts = np.zeros(num_actions)

    def select_action(self, state_index):
        if np.random.rand() < self.epsilon:
            action_index = np.random.randint(len(self.actions))
        else:
            action_index = np.argmax(self.q_table[state_index])
        self.action_counts[action_index] += 1
        return action_index

    def update(self, state_index, action_index, reward, next_state_index):
        best_next_action = np.argmax(self.q_table[next_state_index])
        td_target = reward + self.gamma * self.q_table[next_state_index, best_next_action]
        td_error = td_target - self.q_table[state_index, action_index]
        self.q_table[state_index, action_index] += self.alpha * td_error

    def decay_epsilon(self, decay_rate, min_epsilon):
        self.epsilon = max(min_epsilon, self.epsilon * decay_rate)


class ReviewerAgent:
    def __init__(self, num_states, num_actions, epsilon=0.75, alpha=0.5, gamma=0.8):
        self.q_table = np.zeros((num_states, num_actions))
        self.epsilon = epsilon
        self.alpha = alpha
        self.gamma = gamma
        self.actions = [
            "O código está correto",
            "O código tem erros de sintaxe",
            "O código não resolve o problema",
            "O código é ineficiente",
        ]
        self.action_counts = np.zeros(num_actions)

    def select_action(self, state_index):
        if np.random.rand() < self.epsilon:
            action_index = np.random.randint(len(self.actions))
        else:
            action_index = np.argmax(self.q_table[state_index])
        self.action_counts[action_index] += 1
        return action_index

    def update(self, state_index, action_index, reward, next_state_index, next_action_index):
        td_target = reward + self.gamma * self.q_table[next_state_index, next_action_index]
        td_error = td_target - self.q_table[state_index, action_index]
        self.q_table[state_index, action_index] += self.alpha * td_error

    def decay_epsilon(self, decay_rate, min_epsilon):
        self.epsilon = max(min_epsilon, self.epsilon * decay_rate)


In [6]:
import numpy as np
import pandas as pd

class DatasetDisruptor:
    def __init__(self, seed=None):
        self.seed = seed
        if seed is not None:
            np.random.seed(seed)

    def introduce_missing_values(self, df, missing_fraction=0.1):
        df_broken = df.copy()
        n_total = df.size
        n_missing = int(n_total * min(missing_fraction, 0.2))
        indices = [(row, col) for row in range(df.shape[0]) for col in range(df.shape[1])]
        if n_missing > 0:
            missing_indices = np.random.choice(len(indices), n_missing, replace=False)
            for idx in missing_indices:
                row, col = indices[idx]
                df_broken.iat[row, col] = np.nan
        return df_broken

    def introduce_column_missing(self, df, col_missing_fraction=0.1):
        df_broken = df.copy()
        n_cols = df.shape[1]
        n_cols_missing = int(n_cols * min(col_missing_fraction, 0.1))
        if n_cols_missing > 0:
            cols_to_break = np.random.choice(df.columns, size=n_cols_missing, replace=False)
            df_broken[cols_to_break] = np.nan
        return df_broken

    def introduce_row_missing(self, df, row_missing_fraction=0.05):
        df_broken = df.copy()
        n_rows = df.shape[0]
        n_rows_missing = int(n_rows * min(row_missing_fraction, 0.05))
        if n_rows_missing > 0:
            rows_to_break = np.random.choice(df.index, size=n_rows_missing, replace=False)
            df_broken.loc[rows_to_break] = np.nan
        return df_broken

    def add_noise(self, df, noise_fraction=0.02, noise_level=0.05):
        df_broken = df.copy()
        numeric_cols = df.select_dtypes(include=[np.number]).columns
        n_total = df[numeric_cols].size
        n_noisy = int(n_total * min(noise_fraction, 0.02))

        if n_noisy > 0:
            indices = [(row, col) for row in df.index for col in numeric_cols]
            noisy_indices = np.random.choice(len(indices), n_noisy, replace=False)
            for idx in noisy_indices:
                row, col = indices[idx]
                original_value = df_broken.at[row, col]
                if pd.notnull(original_value):
                    df_broken.at[row, col] += np.random.normal(0, noise_level)
        return df_broken

    def break_dataset(self, df, missing_fraction=0.1, col_missing_fraction=0.1, row_missing_fraction=0.05, noise_fraction=0.02, noise_level=0.05):
        df_broken = self.introduce_missing_values(df, missing_fraction)
        df_broken = self.introduce_column_missing(df_broken, col_missing_fraction)
        df_broken = self.introduce_row_missing(df_broken, row_missing_fraction)
        df_broken = self.add_noise(df_broken, noise_fraction, noise_level)
        return df_broken


In [7]:
class LLMClient:
    def __init__(self, api_key):
        genai.configure(api_key=api_key)

    def generate_code(self, prompt):
        model = genai.GenerativeModel("gemini-1.5-flash")
        response = model.generate_content(prompt)
        code = response.text
        return code.strip()

    def generate_feedback(self, prompt):
        model = genai.GenerativeModel("gemini-1.5-flash")
        response = model.generate_content(prompt)
        feedback = response.text
        return feedback.strip()


In [8]:
def prepare_broken_data():
    disruptor = DatasetDisruptor()

    data_files = [f for f in os.listdir(data_folder) if f.endswith('.csv')]
    if len(data_files) == 0:
        df_example = pd.DataFrame({
            "A":[1,2,3,4,5],
            "B":[10,20,30,40,50],
            "C":[100,200,300,400,500]
        })
        df_example.to_csv(os.path.join(data_folder, "example.csv"), index=False)
        data_files = ["example.csv"]

    broken_files = [f for f in os.listdir(broken_data_folder) if f.endswith('.csv')]
    if len(broken_files) == 0:
        for filename in data_files:
            filepath = os.path.join(data_folder, filename)
            df = pd.read_csv(filepath)
            df_broken = disruptor.break_dataset(df)
            output_filepath = os.path.join(broken_data_folder, filename)
            df_broken.to_csv(output_filepath, index=False)


In [9]:
class DataCleaningEnv:
    def __init__(self, llm_client, data_folder='broken_data'):
        self.llm_client = llm_client
        self.data_folder = data_folder
        self.data_files = [os.path.join(data_folder, f) for f in os.listdir(data_folder) if f.endswith('.csv')]
        if len(self.data_files) == 0:
            raise FileNotFoundError("Nenhum arquivo encontrado em 'broken_data'. Rode prepare_broken_data() primeiro.")
        self.current_file_index = -1
        self.reset()

    def reset(self):
        self.current_file_index = (self.current_file_index + 1) % len(self.data_files)
        data_file = self.data_files[self.current_file_index]
        self.df = pd.read_csv(data_file)
        self.original_df = self.df.copy()
        self.state = self._get_state()
        self.missing_percent_before_feedback = self.df.isnull().mean().mean()
        return self.state

    def _get_state(self):
        missing_percent = self.df.isnull().mean().mean()
        if np.isnan(missing_percent):
            missing_percent = 1.0
        return np.array([missing_percent])

    def step_coder(self, code, action_index):
        fallback_actions = {
            0: (
                "if not df.dropna().empty:\n"
                "    df.dropna(inplace=True)\n"
                "else:\n"
                "    pass"
            ),
            1: (
                "numeric_cols = df.select_dtypes(include=[np.number]).columns\n"
                "df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].mean())"
            ),
            2: "df.fillna(0, inplace=True)",
            3: (
                "threshold = int(0.8 * df.shape[0])\n"
                "df.dropna(axis=1, thresh=threshold, inplace=True)"
            ),
        }

        try:
            if 'input(' in code or 'import os' in code:
                raise ValueError("O código gerado contém chamadas não permitidas.")
            safe_globals = {'df': self.df.copy(), 'pd': pd, 'np': np, '__builtins__': {}}
            safe_locals = {}
            exec(compile(code, '<string>', 'exec'), safe_globals, safe_locals)

            if 'df' in safe_globals:
                self.df = safe_globals['df']
            elif 'df' in safe_locals:
                self.df = safe_locals['df']

            reward = self._calculate_reward_coder()
            done = self._check_done()
            return self._get_state(), reward, done, {}
        except Exception:
            fallback_code = fallback_actions.get(action_index, "")
            exec(fallback_code, {'df': self.df, 'pd': pd, 'np': np})
            reward = self._calculate_reward_coder()
            done = self._check_done()
            return self._get_state(), reward, done, {}

    def step_reviewer(self, feedback):
        missing_percent_after = self.df.isnull().mean().mean()
        if np.isnan(missing_percent_after):
            missing_percent_after = 1.0

        if missing_percent_after < self.missing_percent_before_feedback:
            reward = (self.missing_percent_before_feedback - missing_percent_after) * 10
        elif self.df.empty or self.df.isnull().all().all():
            reward = -1.0
        else:
            reward = -0.5

        self.missing_percent_before_feedback = missing_percent_after
        done = self._check_done()
        return self._get_state(), reward, done, {}

    def _calculate_reward_coder(self):
        if self.df.empty or self.df.isnull().all().all():
            reward = -1.0
        else:
            missing_percent = self.df.isnull().mean().mean()
            reward = -missing_percent
        return reward

    def _check_done(self):
        if self.df.empty or self.df.isnull().all().all():
            return True
        else:
            return False

    def generate_analytic_report(self):
        prompt = """
        Você é um assistente de dados. Com base no DataFrame 'df' atual e no processo de limpeza que foi realizado,
        gere um relatório analítico com as seguintes seções:

        1. Descrição do Problema
        2. Descrição dos Dados
        3. Metodologia
        4. Resultados
        5. Conclusão

        Seja claro, preciso e siga as melhores práticas de análise de dados.
        """
        report = self.llm_client.generate_feedback(prompt)
        return report

    def evaluate_report(self, report):
        prompt = f"""
        Avalie o seguinte relatório em cada um dos critérios listados.
        Retorne as notas de forma estruturada (formato JSON), com cada critério tendo uma nota de 0 a 10.
        Critérios:
        - Descrição do Problema: Clareza, Acurácia
        - Descrição dos Dados: Completude, Qualidade_dados, Visualizacao
        - Metodologia: Abordagem, Justificativa, Implementacao
        - Resultados: Precisao, Compreensao, Visualizacao
        - Conclusao: Resumo, Implicacoes, Recomendacoes

        Relatório:
        {report}
        """

        feedback = self.llm_client.generate_feedback(prompt)
        try:
            scores = json.loads(feedback)
        except:
            scores = {
                "descricao_problema": {"clareza":5,"acuracia":5},
                "descricao_dados": {"completude":5,"qualidade_dados":5,"visualizacao":5},
                "metodologia": {"abordagem":5,"justificativa":5,"implementacao":5},
                "resultados": {"precisao":5,"compreensao":5,"visualizacao":5},
                "conclusao": {"resumo":5,"implicacoes":5,"recomendacoes":5}
            }

        total_score = (scores["descricao_problema"]["clareza"] +
                       scores["descricao_problema"]["acuracia"] +
                       scores["descricao_dados"]["completude"] +
                       scores["descricao_dados"]["qualidade_dados"] +
                       scores["descricao_dados"]["visualizacao"] +
                       scores["metodologia"]["abordagem"] +
                       scores["metodologia"]["justificativa"] +
                       scores["metodologia"]["implementacao"] +
                       scores["resultados"]["precisao"] +
                       scores["resultados"]["compreensao"] +
                       scores["resultados"]["visualizacao"] +
                       scores["conclusao"]["resumo"] +
                       scores["conclusao"]["implicacoes"] +
                       scores["conclusao"]["recomendacoes"])

        return total_score

    def finalize_evaluation(self):
        report = self.generate_analytic_report()
        total_score = self.evaluate_report(report)
        normalized_score = total_score / 150.0
        threshold_score = 135
        done = total_score >= threshold_score
        return normalized_score, done


In [10]:
def discretize_state(state, num_states):
    state_value = state[0]
    if np.isnan(state_value):
        state_value = 1.0
    state_value = min(max(state_value, 0), 0.999)
    return int(state_value * num_states)


In [11]:
def train_agents(api_key, num_episodes=30, decay_rate=0.995, min_epsilon=0.1, max_steps_per_episode=50):
    llm_client = LLMClient(api_key=api_key)
    env = DataCleaningEnv(llm_client, data_folder=broken_data_folder)

    num_states = 10
    coder = CoderAgent(num_states, len(CoderAgent(0,0).actions))
    reviewer = ReviewerAgent(num_states, len(ReviewerAgent(0,0).actions))

    best_coder_reward = float('-inf')
    best_reviewer_reward = float('-inf')

    coder_rewards = []
    reviewer_rewards = []

    start_time = time.time()

    for episode in tqdm(range(num_episodes), desc="Treinando", unit="ep"):
        state = env.reset()
        state_index = discretize_state(state, num_states)
        done = False
        total_coder_reward = 0
        total_reviewer_reward = 0

        coder_action_index = coder.select_action(state_index)
        reviewer_action_index = reviewer.select_action(state_index)

        step_count = 0

        while not done and step_count < max_steps_per_episode:
            step_count += 1

            action_coder = coder.actions[coder_action_index]
            prompt = f"""
            Você é um assistente que ajuda a limpar dataframes do pandas.
            O dataframe 'df' contém dados com valores faltantes e possivelmente ruídos.
            Sua tarefa é: {action_coder}.
            Forneça apenas o código Python necessário para realizar essa tarefa no dataframe 'df'.
            Não inclua explicações ou uso de 'input()' ou 'import os'.
            """
            code = llm_client.generate_code(prompt)

            next_state, reward_coder, done_coder, _ = env.step_coder(code, coder_action_index)
            total_coder_reward += reward_coder
            next_state_index = discretize_state(next_state, num_states)
            next_coder_action_index = coder.select_action(next_state_index)
            coder.update(state_index, coder_action_index, reward_coder, next_state_index)

            action_reviewer = reviewer.actions[reviewer_action_index]
            prompt_feedback = f"""
            O código fornecido foi:
            {code}

            Dê feedback: {action_reviewer}.
            """
            feedback = llm_client.generate_feedback(prompt_feedback)

            next_state_reviewer, reward_reviewer, done_reviewer, _ = env.step_reviewer(feedback)
            total_reviewer_reward += reward_reviewer
            next_reviewer_state_index = discretize_state(next_state_reviewer, num_states)
            next_reviewer_action_index = reviewer.select_action(next_reviewer_state_index)
            reviewer.update(state_index, reviewer_action_index, reward_reviewer, next_reviewer_state_index, next_reviewer_action_index)

            state_index = next_state_index
            coder_action_index = next_coder_action_index
            reviewer_action_index = next_reviewer_action_index

            done = done_coder or done_reviewer

        coder.decay_epsilon(decay_rate, min_epsilon)
        reviewer.decay_epsilon(decay_rate, min_epsilon)

        coder_rewards.append(total_coder_reward)
        reviewer_rewards.append(total_reviewer_reward)

        if total_coder_reward > best_coder_reward:
            best_coder_reward = total_coder_reward
            np.save(os.path.join(models_folder, 'best_coder_q_table.npy'), coder.q_table)

        if total_reviewer_reward > best_reviewer_reward:
            best_reviewer_reward = total_reviewer_reward
            np.save(os.path.join(models_folder, 'best_reviewer_q_table.npy'), reviewer.q_table)

    np.save(os.path.join(models_folder, 'final_coder_q_table.npy'), coder.q_table)
    np.save(os.path.join(models_folder, 'final_reviewer_q_table.npy'), reviewer.q_table)

    elapsed_time = time.time() - start_time
    print(f"Tempo total de treinamento: {elapsed_time:.2f} segundos")

    plt.plot(np.arange(num_episodes), coder_rewards, label='Codificador')
    plt.plot(np.arange(num_episodes), reviewer_rewards, label='Revisor')
    plt.xlabel('Episódios')
    plt.ylabel('Recompensa')
    plt.legend()
    plt.title('Recompensas ao longo dos Episódios')
    plt.savefig(os.path.join(logs_folder, 'recompensas.png'))
    plt.close()

    print("Distribuição de ações do Codificador após o treinamento:")
    for i, action_name in enumerate(coder.actions):
        print(f"Ação {i} ({action_name}): {coder.action_counts[i]} seleções")

    print("Distribuição de ações do Revisor após o treinamento:")
    for i, action_name in enumerate(reviewer.actions):
        print(f"Ação {i} ({action_name}): {reviewer.action_counts[i]} seleções")

    final_score, task_complete = env.finalize_evaluation()
    print(f"Pontuação final do relatório: {final_score*150:.2f} de 150")
    print("Tarefa concluída?" , "Sim" if task_complete else "Não")


In [12]:
SUA_CHAVE_API = " "

prepare_broken_data()

train_agents(api_key=SUA_CHAVE_API, num_episodes=10, decay_rate=0.995, min_epsilon=0.1, max_steps_per_episode=10)


<ipython-input-8-39ae2e1b8dcf>:20: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)
<ipython-input-6-cd806d6eca4f>:19: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'nan' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df_broken.iat[row, col] = np.nan
Treinando:  10%|█         | 1/10 [00:55<08:16, 55.13s/ep]<ipython-input-9-4888254babd3>:14: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  self.df = pd.read_csv(data_file)
Treinando:  60%|██████    | 6/10 [05:06<03:18, 49.69s/ep]<ipython-input-9-4888254babd3>:14: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  self.df = pd.read_csv(data_file)
Treinando: 100%|██████████| 10/10 [08:46<00:00, 52.67s/ep]


Tempo total de treinamento: 526.75 segundos
Distribuição de ações do Codificador após o treinamento:
Ação 0 (Remover linhas com valores nulos em 'df'): 52.0 seleções
Ação 1 (Preencher valores nulos em colunas numéricas de 'df' com a média): 17.0 seleções
Ação 2 (Preencher valores nulos em 'df' com zero): 20.0 seleções
Ação 3 (Remover colunas com muitos valores nulos em 'df'): 21.0 seleções
Distribuição de ações do Revisor após o treinamento:
Ação 0 (O código está correto): 23.0 seleções
Ação 1 (O código tem erros de sintaxe): 26.0 seleções
Ação 2 (O código não resolve o problema): 36.0 seleções
Ação 3 (O código é ineficiente): 25.0 seleções
Pontuação final do relatório: 70.00 de 150
Tarefa concluída? Não
